<h1>  Missionaries and Cannibals Problem </h1>

<p> Missionaries and Cannibals is a problem in which 3 missionaries and 3 cannibals want to
cross from the left bank of a river to the right bank of the river. There is a boat on the left
bank, but it only carries at most two people at a time (and can never cross with zero
people). If cannibals ever outnumber missionaries on either bank, the cannibals will eat
the missionaries. </p>


In [2]:
import time
import pandas as pd

boat_from_2_to_1=["M","MM","C","CC","MC"]
boat_from_1_to_2=["MM","CC","MC"]

class State:
    def __init__(self, missionaries, cannibals):
        self.missionaries = missionaries
        self.missionaries_left = missionaries
        self.cannibals = cannibals
        self.cannibals_left = cannibals

    def isvalid(self):
        if (self.missionaries_left <= 3 and self.missionaries_left >= 0 and
                self.cannibals_left <= 3 and self.cannibals_left >= 0):
            if self.missionaries_left == 0:
                return True
            elif self.cannibals_left <= self.missionaries_left:
                return True
            else:
                return False
        else:
            return False

    def departure(self, who):
        for i in list(who):
            if i == "M":
                self.missionaries_left -= 1
            if i == "C":
                self.cannibals_left -= 1
        m,c=(self.missionaries_left, self.cannibals_left)
        self.missionaries_left=self.missionaries
        self.cannibals_left=self.cannibals
        return (m,c)

    def arrival(self, who):
        for i in list(who):
            if i == "M":
                self.missionaries_left += 1
            if i == "C":
                self.cannibals_left += 1
        m, c = (self.missionaries_left, self.cannibals_left)
        self.missionaries_left = self.missionaries
        self.cannibals_left = self.cannibals
        return (m,c)



class Root:

    def __init__(self, Land1_missionaries, Land1_cannibals, Land2_missionaries, Land2_cannibals,boat_position,cost):
        self.Land1 = State(Land1_missionaries, Land1_cannibals)
        self.Land2 = State(Land2_missionaries, Land2_cannibals)
        self.exist = False
        self.boat_position=boat_position
        self.cost=cost
        self.previous=None

    def doexist(self):
        if self.Land1.isvalid() and self.Land2.isvalid():
            self.exist = True


<h5>  Breadth-first search </h5>

In [3]:
def BFS(root):
    visited = []
    visited.append([root.Land1.missionaries, root.Land1.cannibals,
                    root.Land2.missionaries, root.Land2.cannibals,root.boat_position])
    runner = []
    cost=0
    runner1=[]
    runner1.append(root)
    while not (len(runner) == 0 and len(runner1)==0):
        while not(len(runner1)==0):
            root = runner1.pop(0)
            for i in boat_from_1_to_2:
                missionary1, cannibal1 = root.Land1.departure(i)
                missionary2, cannibal2 = root.Land2.arrival(i)
                node = Root(missionary1, cannibal1, missionary2, cannibal2,1,root.cost+1)
                node.doexist()
                if node.exist:
                    if not ([node.Land1.missionaries, node.Land1.cannibals,
                             node.Land2.missionaries, node.Land2.cannibals,node.boat_position] in visited):
                        node.previous=root
                        runner.append(node)
                        visited.append([node.Land1.missionaries, node.Land1.cannibals,
                                        node.Land2.missionaries, node.Land2.cannibals,node.boat_position])
                        if node.Land1.missionaries == 0 and node.Land1.cannibals == 0 and node.Land2.missionaries == 3 and node.Land2.cannibals == 3:
                            return node

        while not(len(runner)==0):
            root=runner.pop(0)
            for j in boat_from_2_to_1:
                missionary2, cannibal2 = root.Land2.departure(j)
                missionary1, cannibal1 = root.Land1.arrival(j)
                node = Root(missionary1, cannibal1, missionary2, cannibal2,0,root.cost+1)
                node.doexist()
                if node.exist:
                    if not ([node.Land1.missionaries, node.Land1.cannibals,
                             node.Land2.missionaries, node.Land2.cannibals,node.boat_position] in visited):
                        node.previous=root
                        runner1.append(node)
                        visited.append([node.Land1.missionaries, node.Land1.cannibals,
                                        node.Land2.missionaries, node.Land2.cannibals,node.boat_position])
                        if node.Land1.missionaries == 0 and node.Land1.cannibals ==0 and node.Land2.missionaries==3 and node.Land2.cannibals==3:
                            return node

    return visited,cost

<h5>  Depth-first search </h5>

In [4]:
def DFS_visit(root,visited):
    for i in boat_from_1_to_2:
        missionary1, cannibal1 = root.Land1.departure(i)
        missionary2, cannibal2 = root.Land2.arrival(i)
        node = Root(missionary1, cannibal1, missionary2, cannibal2, 1, root.cost + 1)
        node.doexist()

        if node.exist:
            if not ([node.Land1.missionaries, node.Land1.cannibals,
                     node.Land2.missionaries, node.Land2.cannibals, node.boat_position] in visited):
                node.previous=root
                visited.append([node.Land1.missionaries, node.Land1.cannibals,
                                node.Land2.missionaries, node.Land2.cannibals, node.boat_position])
                if node.Land1.missionaries == 0 and node.Land1.cannibals == 0 and node.Land2.missionaries == 3 and node.Land2.cannibals == 3:
                    return node

                for j in boat_from_2_to_1:
                    missionary21, cannibal22 = node.Land2.departure(j)
                    missionary11, cannibal12 = node.Land1.arrival(j)
                    node1 = Root(missionary11, cannibal12, missionary21, cannibal22, 0, node.cost + 1)
                    node1.doexist()
                    if node1.exist:
                        if not ([node1.Land1.missionaries, node1.Land1.cannibals,
                                 node1.Land2.missionaries, node1.Land2.cannibals, node1.boat_position] in visited):
                            node1.previous=node
                            visited.append([node1.Land1.missionaries, node1.Land1.cannibals,
                                            node1.Land2.missionaries, node1.Land2.cannibals, node1.boat_position])
                            if node1.Land1.missionaries == 0 and node1.Land1.cannibals == 0 and node1.Land2.missionaries == 3 and node1.Land2.cannibals == 3:
                                return node1

                            return DFS_visit(node1,visited)


def DFS(root):
    visited=[]
    visited.append([root.Land1.missionaries, root.Land1.cannibals,
                    root.Land2.missionaries, root.Land2.cannibals,root.boat_position])
    return DFS_visit(root,visited)


<h5> Dijkstra's Algorithm </h5>

In [5]:
def Dijkstras(root):
    runner={}
    runner[root.cost]=[root]
    visited=[]
    Costs=[root.cost]

    while not len(runner)==0:
        root_list = runner.pop(Costs[0])
        Costs.pop(0)
        for root in root_list:
            if  ([root.Land1.missionaries, root.Land1.cannibals,
                     root.Land2.missionaries, root.Land2.cannibals, root.boat_position] in visited):
                continue

            elif root.boat_position==0:
                for i in boat_from_1_to_2:
                    missionary1, cannibal1 = root.Land1.departure(i)
                    missionary2, cannibal2 = root.Land2.arrival(i)
                    node = Root(missionary1, cannibal1, missionary2, cannibal2, 1, root.cost + 1)
                    node.doexist()
                    if node.exist:
                        if not ([node.Land1.missionaries, node.Land1.cannibals,
                                 node.Land2.missionaries, node.Land2.cannibals, node.boat_position]in visited):
                            if not node.cost in runner:
                                node.previous=root
                                runner[node.cost]=[node]
                                Costs.append(node.cost)
                            else:
                                node.previous=root
                                runner[node.cost].append(node)
                            if node.Land1.missionaries == 0 and node.Land1.cannibals == 0 and node.Land2.missionaries == 3 and node.Land2.cannibals == 3:
                                return node

                visited.append([root.Land1.missionaries, root.Land1.cannibals,
                            root.Land2.missionaries, root.Land2.cannibals, root.boat_position])

            else:
                for j in boat_from_2_to_1:
                    missionary2, cannibal2 = root.Land2.departure(j)
                    missionary1, cannibal1 = root.Land1.arrival(j)
                    node = Root(missionary1, cannibal1, missionary2, cannibal2, 0, root.cost + 1)
                    node.doexist()
                    if node.exist:
                        if not ([node.Land1.missionaries, node.Land1.cannibals,
                                 node.Land2.missionaries, node.Land2.cannibals, node.boat_position] in visited):
                            if not node.cost in runner:
                                node.previous=root
                                runner[node.cost]=[node]
                                Costs.append(node.cost)
                            else:
                                node.previous=root
                                runner[node.cost].append(node)
                            if node.Land1.missionaries == 0 and node.Land1.cannibals == 0 and node.Land2.missionaries == 3 and node.Land2.cannibals == 3:
                                return node
                visited.append([root.Land1.missionaries, root.Land1.cannibals,
                        root.Land2.missionaries, root.Land2.cannibals, root.boat_position])
        Costs.sort()
    return visited

In [6]:
root=Root(3,3,0,0,0,0)

def Path(root,solver):
    node=solver(root)
    df={"Land1 M":[],"Land1 C":[],"Land2 M":[],"Land2 C":[],"BOAT Position":[]}
    while not node.previous==None:
        df["Land1 M"].append(node.Land1.missionaries)
        df["Land1 C"].append(node.Land1.cannibals)
        df["Land2 M"].append(node.Land2.missionaries)
        df["Land2 C"].append(node.Land2.cannibals)
        df["BOAT Position"].append(node.boat_position)
        node=node.previous
    
    data = pd.DataFrame(df)
    data = data.reindex(index=data.index[::-1])
    print(data)
    print("\nNumber of actions required: ", data["Land1 M"].count())


In [7]:
start = time.time()
Path(root,DFS)
print("\nExecution time for DFS is: ", (time.time() - start) )

    Land1 M  Land1 C  Land2 M  Land2 C  BOAT Position
10        3        1        0        2              1
9         3        2        0        1              0
8         3        0        0        3              1
7         3        1        0        2              0
6         1        1        2        2              1
5         2        2        1        1              0
4         0        2        3        1              1
3         0        3        3        0              0
2         0        1        3        2              1
1         1        1        2        2              0
0         0        0        3        3              1

Number of actions required:  11

Execution time for DFS is:  0.00961160659790039


In [8]:
start = time.time()
Path(root,BFS)
print("\nExecution time for BFS is: ", (time.time() - start) )

    Land1 M  Land1 C  Land2 M  Land2 C  BOAT Position
10        3        1        0        2              1
9         3        2        0        1              0
8         3        0        0        3              1
7         3        1        0        2              0
6         1        1        2        2              1
5         2        2        1        1              0
4         0        2        3        1              1
3         0        3        3        0              0
2         0        1        3        2              1
1         1        1        2        2              0
0         0        0        3        3              1

Number of actions required:  11

Execution time for BFS is:  0.007502317428588867


In [9]:
# print(Dijkstras(root))
start = time.time()
Path(root,Dijkstras)
print("\nExecution time for BFS is: ", (time.time() - start) )

    Land1 M  Land1 C  Land2 M  Land2 C  BOAT Position
10        3        1        0        2              1
9         3        2        0        1              0
8         3        0        0        3              1
7         3        1        0        2              0
6         1        1        2        2              1
5         2        2        1        1              0
4         0        2        3        1              1
3         0        3        3        0              0
2         0        1        3        2              1
1         1        1        2        2              0
0         0        0        3        3              1

Number of actions required:  11

Execution time for BFS is:  0.008213520050048828
